In [165]:
import pandas as pd
import os
import sys

sys.path.append('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

pathPrefix = '/Users/friedman/Desktop/mnt'

from collections import Counter

In [4]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')



In [6]:
len(gliomaIds), len(colorectalIds), len(endometrialIds)

(53, 261, 237)

In [7]:
allImpactMuts = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
hypermutantMutationsDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalIds | endometrialIds | gliomaIds)]

In [45]:
hypermutantMutationsDf['pid'] = hypermutantMutationsDf['Tumor_Sample_Barcode'].apply(lambda x: x[:9])
s = hypermutantMutationsDf.drop_duplicates(subset=['Tumor_Sample_Barcode'])['pid'].value_counts()
casesWithMultiples = s[s != 1].index
casesWithThreeSamples = s[s == 3].index
casesWithTwoSamples = s[s == 2].index

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [147]:
multipleSampleTumorSampleBarcodes =  set(['P-0001882-T02-IM5', 'P-0001882-T03-IM6']) | set(hypermutantMutationsDf[hypermutantMutationsDf['pid'].isin(casesWithTwoSamples)]['Tumor_Sample_Barcode'])
#WE ARE GOING TO GET RID OF THE CASES that are not actually related (MANUAL CURATION)
multipleSampleTumorSampleBarcodes = multipleSampleTumorSampleBarcodes - set(['P-0021519-T02-IM6', 'P-0021519-T01-IM6',
                                                                             'P-0020633-T01-IM6', 'P-0020633-T02-IM6',
                                                                             'P-0000083-T02-IM6', 'P-0000083-T01-IM3',
                                                                            'P-0001685-T02-IM3', 'P-0001685-T01-IM3'])


In [175]:
mutationsWithMultipleSamples = hypermutantMutationsDf[hypermutantMutationsDf['Tumor_Sample_Barcode'].isin(multipleSampleTumorSampleBarcodes)]

In [176]:
reload(maf_analysis_utils)
mutationsWithMultipleSamples = maf_analysis_utils.mark_private_vs_shared_mutations(mutationsWithMultipleSamples)

In [177]:
#Set up the cancer type implication dict info
cancerTypeImplicationDictBig = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.1)
cancerTypeImplicationDictSmall = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.025) 
for key, value in cancerTypeImplicationDictBig.items():
    curCTypeGenes = cancerTypeImplicationDictSmall[key]
    cancerTypeImplicationDictSmall[key] = curCTypeGenes - value

Endometrial Cancer


/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils/maf_analysis_utils.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cohortMaf['pid'] = cohortMaf['Tumor_Sample_Barcode'].apply(lambda x: x[:9])
/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils/maf_analysis_utils.py:395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  oncogenicMutations['patientGeneMutated'] = oncogenicMutations.apply(lambda row: row['pid'] + '_' + row['Hugo_Symbol'], axis=1)


Glioma
Colorectal Cancer
Endometrial Cancer
Glioma
Colorectal Cancer


In [178]:
#add cancer type info to the maf
cDict = analysis_utils.get_cancer_type_information(cancerTypeDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/mskImpactAsOfMarch2019/dmp/mskimpact/data_clinical_sample.txt')
mutationsWithMultipleSamples['cancer_type'] = mutationsWithMultipleSamples['pid'].apply(lambda x: cDict[x] if x in cDict else None)
#we need to adjust some cancer type names
renameMapping = {'Pleural Mesothelioma, Epithelioid Type': 'Mesothelioma','Breast Invasive Ductal Carcinoma': 'Breast Cancer','Bladder Urothelial Carcinoma': 'Bladder Cancer','Upper Tract Urothelial Carcinoma': 'Bladder Cancer',
'Colon Adenocarcinoma': 'Colorectal Cancer', 'Glioblastoma Multiforme': 'Glioma','Adenocarcinoma of the Gastroesophageal Junction': 'Esophagogastric Cancer','Pancreatic Neuroendocrine Tumor': 'Pancreatic Cancer',
'Endometrial Carcinoma': 'Endometrial Cancer','Stomach Adenocarcinoma': 'Esophagogastric Cancer','Rectal Adenocarcinoma': 'Colorectal Cancer','High-Grade Serous Ovarian Cancer': 'Ovarian Cancer','Breast Invasive Lobular Carcinoma': 'Breast Cancer',
'Oligodendroglioma': 'Glioma','Serous Ovarian Cancer': 'Ovarian Cancer','Prostate Adenocarcinoma': 'Prostate Cancer','Breast Invasive Carcinoma, NOS': 'Breast Cancer','Esophageal Adenocarcinoma': 'Esophagogastric Cancer',
'Invasive Breast Carcinoma': 'Breast Cancer','Pancreatic Adenocarcinoma': 'Pancreatic Cancer','Uterine Endometrioid Carcinoma': 'Endometrial Cancer','Colorectal Adenocarcinoma': 'Colorectal Cancer','Mucinous Adenocarcinoma of the Colon and Rectum': 'Colorectal Cancer'
}
mutationsWithMultipleSamples['cancer_type'] = mutationsWithMultipleSamples['cancer_type'].apply(lambda x: renameMapping[x] if x in renameMapping else x)





/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [179]:
mutationsWithMultipleSamples = maf_analysis_utils.mark_mutations_by_gene_mut_type(mutationsWithMultipleSamples, cancerTypeImplicationDictBig, cancerTypeImplicationDictSmall)


In [208]:
listOfDicts = []
def summarize_mutation_info_for_plotting(maf):
    for case in set(maf['Tumor_Sample_Barcode']):
        caseMafOnc = maf[(maf['Tumor_Sample_Barcode'] == case) & maf['oncogenic'].notnull()]
        sharedMuts = caseMafOnc[(caseMafOnc['isSharedMut'] == True)]
        privateMuts = caseMafOnc[(caseMafOnc['isSharedMut'] == False)]
        
        nOncShared = sharedMuts.shape[0] + 0.00001
        nOncPrivate = privateMuts.shape[0] + 0.000001
        
        #inefficient homesizzle
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'OncSharedStrong', 'timing' : 'early', 'orderingVal': 1,
            'n': 1.0*sharedMuts[sharedMuts['geneMutType'] == 'strongly_recurrent'].shape[0]/nOncShared})
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'nOncPrivateStrong', 'timing' : 'late', 'orderingVal': 2,
            'n': 1.0*privateMuts[privateMuts['geneMutType'] == 'strongly_recurrent'].shape[0]/nOncPrivate})
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'nOncSharedWeak', 'timing' : 'early', 'orderingVal': 3,
            'n': 1.0*sharedMuts[sharedMuts['geneMutType'] == 'weakly_recurrent'].shape[0]/nOncShared})
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'nOncPrivateWeak', 'timing' : 'late', 'orderingVal': 4,
            'n': 1.0*privateMuts[privateMuts['geneMutType'] == 'weakly_recurrent'].shape[0]/nOncPrivate})
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'nOncSharedUnrelated', 'timing' : 'early', 'orderingVal': 5,
            'n': 1.0*sharedMuts[sharedMuts['geneMutType'] == 'not_recurrent'].shape[0]/nOncShared})
        listOfDicts.append({
            'Tumor_Sample_Barcode': case, 'type': 'nOncPrivateUnrelated', 'timing' :'late', 'orderingVal': 6, 
            'n': 1.0*privateMuts[privateMuts['geneMutType'] == 'not_recurrent'].shape[0]/nOncPrivate})

    return pd.DataFrame(listOfDicts)
        
        

In [209]:
df = summarize_mutation_info_for_plotting(mutationsWithMultipleSamples)

In [210]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/privateVsSharedMutType.tsv', index=False, sep='\t')